# Data transformation

* 23 April 2021
    * Cleaned columns
        * publisher: removed whitespace
        * title: capitalized columns and removed whitespace
        * author: removed whitespace
        * contributor: removed 'by' and whitespace
    * Created functions 
        * first_name: Author's first name
        * last_name: Author's last name


* 24 April
    * Renamed the 'nyt_bestsellers.csv' file to 'nyt_hardcover_fiction.csv'
    * Merged the 'nyt_hardcover_fiction.csv' and 'nyt_hardcover_nonfiction.csv' files
    * Created a list of authors
    * Cleaned ''James O Born' and be consistent
    * Created subsets csv files of the data to load into MySQL database


* 29 April 2021
    * Created the the dataframe of the books, with 'publisher_id' and 'author_id'.
    * Created a dataframe with the list entries
    * Rename named 'published_date' to 'date_published'
    
    
* 1 May 2021
    * Decided to drop the 'age_group' column because it does not have any values.
    * Fine tuned the order of all of the dataframes to ensure accurate csv files for database loading.

In [1]:
#Loading the libraries
import pandas as pd

In [2]:
#Loading the dataframes
fiction = pd.read_csv('nyt_hardcover_fiction.csv')
nonfiction = pd.read_csv('nyt_hardcover_nonfiction.csv')
nyt_lists = pd.read_csv('lists.csv')

#Merging the dataframes
df = pd.concat([fiction, nonfiction])
df.head()

,weeks_on_list,publisher,title,author,contributor,age_group,published_date,list_name
0,1,Putnam,OCEAN PREY,John Sandford,by John Sandford,NaN,2021-05-02,Hardcover Fiction
1,3,Viking,THE HILL WE CLIMB,Amanda Gorman,by Amanda Gorman,NaN,2021-05-02,Hardcover Fiction
2,1,Emily Bestler/Atria,THE DEVIL'S HAND,Jack Carr,by Jack Carr,NaN,2021-05-02,Hardcover Fiction
3,11,St. Martin's,THE FOUR WINDS,Kristin Hannah,by Kristin Hannah,NaN,2021-05-02,Hardcover Fiction
4,20,Viking,THE MIDNIGHT LIBRARY,Matt Haig,by Matt Haig,NaN,2021-05-02,Hardcover Fiction


In [3]:
#Renaming 
df.rename(columns = {'published_date': 'date_published'}, inplace = True)

In [4]:
#Checking the shape of the dataframe
df.shape

(2100, 8)

In [5]:
#Checking for null values
df.isnull().sum()

weeks_on_list        0
publisher            0
title                0
author               0
contributor          0
age_group         2100
date_published       0
list_name            0
dtype: int64

In [6]:
df['age_group'] = '' #Changed to make database loading easier

In [7]:
#Cleaning the data
df['publisher'] = df.publisher.str.strip()
df['title'] = df.title.str.capitalize().str.strip()
df['author'] = df.author.str.strip()
df['contributor'] = df.contributor.str.replace('by', '').str.strip()

In [8]:
#The author 'James O. Born' has two version 
#of thier name that have to be fixed
df.loc[df.author.str.lower().str.contains('o born'), 'author'] = df.loc[df.author.str.lower().str.contains('o born'), 'author'].str.replace('James O Born', 'James O. Born')

In [9]:
#Cleaning the columns with 'edited'
df.loc[df['author'].str.contains('edited'), 'author'] = df.loc[df['author'].str.contains('edited'), 'author'].str.replace('edited', '').str.strip()
df.loc[df['contributor'].str.contains('edited'), 'contributor'] = df.loc[df['contributor'].str.contains('edited'), 'contributor'].str.replace('edited', '')

In [10]:
def small_dataframes(df, column_name, primary_key, new_column_name):
    
    """This function generates a dataframe with a primary 
    key column and the data column"""
    
    df = pd.DataFrame(df[column_name].unique(), columns = [new_column_name])
    df.sort_values(by = new_column_name).reset_index(drop = True, inplace = True)
    new_indices = [i for i in range(1, len(df) + 1)]
    df[primary_key] = new_indices
    df = df[[primary_key, new_column_name]]
    return df

In [11]:
authors = small_dataframes(df, 'author', 'author_id', 'author_name')
authors.to_csv('authors.csv', index = False, encoding = 'utf-8')

publishers = small_dataframes(df, 'publisher', 'publisher_id', 'publisher_name')
publishers.to_csv('publishers.csv', index = False, encoding = 'utf-8')

In [12]:
#Creating the dataframe for the 'books table'
books_temp = df[['title', 'author', 'publisher']].copy()
books_temp.drop_duplicates(['title', 'author'], keep = 'first', inplace = True)
books = pd.merge(authors, books_temp, left_on = 'author_name', right_on = 'author', how = 'inner')
books = pd.merge(books, publishers, left_on = 'publisher', right_on = 'publisher_name', how = 'inner')
indices = [i for i in range(1, len(books) + 1)]
books['book_id'] = indices

In [13]:
#Creating the list entries dataframe
list_entries_columns = [ 'list_entry_id', 'date_published','book_id',
                        'author_id', 'publisher_id', 'contributor', 
                        'list_id', 'weeks_on_list', 'age_group']

df_copy = df.copy()
#Joining on title and author because 
#some books have the same title 
list_entries = pd.merge(df_copy, books, on = ['title', 'author'], how = 'left')  
list_entries = pd.merge(list_entries, nyt_lists, on = 'list_name', how = 'left')
list_entry_indices = [i for i in range(1, len(list_entries) + 1)]
list_entries['list_entry_id'] = list_entry_indices
list_entries = list_entries[list_entries_columns]
list_entries.to_csv('list_entries.csv', index = False, na_rep = r'\N', encoding = 'utf-8')

In [14]:
book_columns = ['book_id', 'title', 'author_id', 'publisher_id']
books = books[book_columns].copy() #Reducing the columns in the books dataframes
books.to_csv('books.csv', index = False, encoding = 'utf-8')